In [11]:
from PIL import Image
import os
import numpy as np
from collections import deque

already_covered = (0, 0, 0)  # Color for covered area
possible_places = (195, 195, 195)  # Color for to cover area
to_cover_next_layer = (0, 255, 0)  # Color for layer itself

height = None
width = None

def load_image(image_path):
    img = Image.open(image_path)
    img = img.convert("RGB")
    img = np.array(img)

    global height, width
    height, width, _ = img.shape
    
    already_covered_coords = np.argwhere(np.all(img == already_covered, axis=-1))


    possible_places_coords = np.argwhere(np.all(img == possible_places, axis=-1))


    to_cover_next_layer_coords = np.argwhere(np.all(img == to_cover_next_layer, axis=-1))


    return set([tuple(i.tolist()) for i in already_covered_coords]), set([tuple(i.tolist()) for i in possible_places_coords]), deque([tuple(i.tolist()) for i in to_cover_next_layer_coords])
    

already_covered_coords, possible_places_coords, to_cover_next_layer_coords = load_image("test.png")

In [12]:
print("Already Covered Coordinates:", already_covered_coords)
print("possible_places_coords:", possible_places_coords)
print("Layer Coordinates:", to_cover_next_layer_coords)

Already Covered Coordinates: set()
possible_places_coords: {(26, 21), (63, 25), (29, 32), (60, 19), (6, 48), (33, 38), (52, 51), (64, 25), (10, 54), (56, 21), (2, 50), (22, 28), (14, 24), (49, 18), (36, 48), (15, 25), (55, 61), (48, 22), (18, 30), (59, 68), (29, 45), (40, 54), (41, 19), (33, 15), (25, 11), (44, 24), (34, 16), (25, 47), (45, 25), (86, 26), (32, 55), (37, 21), (78, 22), (43, 64), (35, 60), (26, 11), (78, 58), (79, 23), (70, 54), (71, 19), (36, 61), (9, 44), (28, 57), (29, 22), (75, 25), (21, 54), (54, 51), (33, 28), (10, 44), (14, 14), (66, 61), (67, 26), (13, 55), (59, 22), (17, 25), (36, 38), (55, 51), (28, 34), (5, 50), (58, 62), (29, 35), (21, 31), (61, 67), (60, 22), (6, 51), (52, 18), (10, 21), (54, 64), (32, 45), (43, 54), (35, 50), (47, 24), (36, 51), (82, 18), (28, 47), (29, 12), (48, 25), (21, 8), (40, 21), (33, 18), (73, 54), (74, 19), (39, 61), (12, 44), (31, 57), (32, 22), (23, 53), (22, 8), (35, 27), (66, 51), (13, 45), (32, 58), (78, 25), (24, 54), (70, 21

In [13]:
for i in range(len(to_cover_next_layer_coords)):
    item=to_cover_next_layer_coords.popleft()
    possible_children = [
        (item[0], item[1] + 1), (item[0], item[1] - 1),
        (item[0] - 1, item[1]), (item[0] + 1, item[1]),
        (item[0] - 1, item[1] - 1), (item[0] - 1, item[1] + 1),
        (item[0] + 1, item[1] - 1), (item[0] + 1, item[1] + 1)
    ]
    for child in possible_children:
        if child in possible_places_coords and child not in already_covered_coords:
            to_cover_next_layer_coords.append(child)
            already_covered_coords.add(child)


In [14]:
print("To Cover Coordinates:", to_cover_next_layer_coords)
print("Already Covered Coordinates:", already_covered_coords)

To Cover Coordinates: deque([(86, 17), (86, 18), (86, 19), (86, 20), (86, 21), (86, 22), (86, 23), (86, 24), (86, 25), (86, 26)])
Already Covered Coordinates: {(86, 21), (86, 24), (86, 20), (86, 17), (86, 23), (86, 26), (86, 22), (86, 19), (86, 25), (86, 18)}


In [205]:
# Create a new image to visualize the current state
if height is None or width is None:
    height = max(max(coord[0] for coord in already_covered_coords), 
                max(coord[0] for coord in possible_places_coords)) + 1
    width = max(max(coord[1] for coord in already_covered_coords), 
                max(coord[1] for coord in possible_places_coords)) + 1

# Create white background
result_img = np.full((height, width, 3), 255, dtype=np.uint8)

# Set colors for each coordinate set
for coord in possible_places_coords:
    result_img[coord[0], coord[1]] = possible_places

for coord in already_covered_coords:
    result_img[coord[0], coord[1]] = already_covered

for coord in to_cover_next_layer_coords:
    result_img[coord[0], coord[1]] = to_cover_next_layer

# Convert to PIL Image and save
result_image = Image.fromarray(result_img)
result_image.save("result.png")
print("Image saved as 'result.png'")

Image saved as 'result.png'


In [206]:
def scale_image_no_interpolation(input_path, output_path, scale_factor):
    """
    Scales an image by an integer factor without interpolation (nearest neighbor).
    Args:
        input_path (str): Path to the input image.
        output_path (str): Path to save the scaled image.
        scale_factor (int): Scaling factor (must be >= 1).
    Returns:
        str: Path to the saved scaled image.
    """
    img = Image.open(input_path)
    img = img.convert("RGB")
    arr = np.array(img)
    scaled_arr = np.repeat(np.repeat(arr, scale_factor, axis=0), scale_factor, axis=1)
    scaled_img = Image.fromarray(scaled_arr)
    scaled_img.save(output_path)
    return output_path

In [207]:
scale_image_no_interpolation("result.png", "scaled_result.png", 10)

'scaled_result.png'

In [220]:
import subprocess
import requests
import time
import ollama

OLLAMA_URL = "http://localhost:11434"

def is_ollama_running():
    try:
        response = requests.get(OLLAMA_URL)
        return response.status_code == 200
    except requests.exceptions.ConnectionError:
        return False

def start_ollama_daemon():
    # This assumes 'ollama' is in your system PATH
    subprocess.Popen(['ollama', 'serve'], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

def pull_model(model_name):
    subprocess.run(['ollama', 'pull', model_name])

def wait_for_server(timeout=15):
    for _ in range(timeout):
        if is_ollama_running():
            return True
        time.sleep(1)
    return False

# --- Main automation flow ---
model = 'qwen2.5vl:7b'

if not is_ollama_running():
    print("Starting Ollama server...")
    start_ollama_daemon()
    if not wait_for_server():
        raise RuntimeError("Ollama server did not start in time.")

print(f"Pulling model '{model}' (if needed)...")
pull_model(model)

print("Sending chat message...")
for i in range(10):
    response = ollama.chat(
        model=model,
        messages=[
            {'role': 'user', 'content': 'What are these images?', 'images': ['./result.png', './scaled_result.png']},
            
        ]
    )

    print("Response:", response['message']['content'])


Pulling model 'qwen2.5vl:7b' (if needed)...
Sending chat message...
Response: The images you've provided appear to be pixel art, which is a style of digital art where images are created using pixels, the smallest units of color in a digital image. The first image seems to depict a stylized figure or character, possibly a robot or a mechanical entity, with a prominent arm or limb extending upwards. The second image is similar but includes additional details, such as green elements near the base of the figure, which could represent a base or platform. Both images are quite abstract and could be interpreted in various ways depending on the context or the artist's intent.
Response: The images you've provided appear to be pixel art, which is a style of digital art where images are created using pixels, the smallest units of color in a digital image. The first image seems to depict a stylized figure or character, possibly a robot or a mechanical entity, with a prominent arm or limb extending

In [17]:
green_groups = [[(78, 59), (79, 59), (79, 58)], [(74, 65), (74, 66), (74, 64), (74, 63), (74, 62)]]

numbered_groups = {i: group for i, group in enumerate(green_groups)}
numbered_groups_string = '\n'+'\n'.join(f"{i}: {group}" for i, group in numbered_groups.items())

print("Numbered Groups:", numbered_groups_string)

Numbered Groups: 
0: [(78, 59), (79, 59), (79, 58)]
1: [(74, 65), (74, 66), (74, 64), (74, 63), (74, 62)]
